In [86]:
from pymol import cmd
import py3Dmol
import prolif as plf
import MDAnalysis as mda
from MDAnalysis.coordinates import PDB
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from openbabel import pybel
from vina import Vina
from meeko import MoleculePreparation
from meeko import obutils
import prolif as plf
from prolif.plotting.network import LigNetwork
import pandas as pd
import numpy as np
from Bio.PDB.PDBParser import PDBParser

In [2]:
import sys, os, random
sys.path.insert(1, 'utilities/')
from utils import fix_protein, getbox, generate_ledock_file, pdbqt_to_sdf, dok_to_sdf

In [3]:
import warnings
warnings.filterwarnings("ignore")
%config Completer.use_jedi = False

In [4]:
os.chdir('successive_runs/')

In [5]:
# Feching system and cleanup
cmd.fetch(code='3QGH',type='pdb1')
cmd.select(name='Prot',selection='Chain A')
cmd.save(filename='rec_clean.pdb',format='pdb',selection='Prot')
cmd.delete('all')

In [340]:
# Residues
structure = PDBParser().get_structure('rec_clean_H', 'rec_clean_H.pdb')    
model = structure[0]
chain = model['A']
for i in chain.get_residues():
    print(i)

<Residue SER het=  resseq=1 icode= >
<Residue MET het=  resseq=2 icode= >
<Residue SER het=  resseq=3 icode= >
<Residue TYR het=  resseq=4 icode= >
<Residue SER het=  resseq=5 icode= >
<Residue TRP het=  resseq=6 icode= >
<Residue THR het=  resseq=7 icode= >
<Residue GLY het=  resseq=8 icode= >
<Residue ALA het=  resseq=9 icode= >
<Residue LEU het=  resseq=10 icode= >
<Residue VAL het=  resseq=11 icode= >
<Residue THR het=  resseq=12 icode= >
<Residue PRO het=  resseq=13 icode= >
<Residue CYS het=  resseq=14 icode= >
<Residue ALA het=  resseq=15 icode= >
<Residue ALA het=  resseq=16 icode= >
<Residue GLU het=  resseq=17 icode= >
<Residue GLU het=  resseq=18 icode= >
<Residue GLN het=  resseq=19 icode= >
<Residue LYS het=  resseq=20 icode= >
<Residue LEU het=  resseq=21 icode= >
<Residue PRO het=  resseq=22 icode= >
<Residue ILE het=  resseq=23 icode= >
<Residue ASN het=  resseq=24 icode= >
<Residue ALA het=  resseq=25 icode= >
<Residue LEU het=  resseq=26 icode= >
<Residue SER het=  re

In [6]:
# System Visualization
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('rec_clean.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
# Receptor preparation
#!../../bin/lepro_linux_x86 
!../bin/lepro_linux_x86 {'rec_clean.pdb'}
os.rename('pro.pdb','rec_clean_H.pdb')

In [8]:
#Ligand preparation
smiles=['C1C(NC(=CC1=CC=[N+]2C(CC3=CC(=C(C=C32)O)OC4C(C(C(C(O4)CO)O)O)O)C(=O)[O-])C(=O)O)C(=O)O',
        'CN1CCC2=C3C1CC4=C(C(=C(C=C4C3=C(C(=C2O)OC)OC)OC)O)OC5=CC=C(C=C5)CC6C7=CC(=C(C(=C7CCN6C)O)OC)OC',
        'COC1=C(C=C(C=C1)CC2COC(=O)C2CC3=CC(=C(C(=C3)OC)O)C4=C(C(=CC(=C4)CC5C(COC5=O)CC6=CC(=C(C=C6)OC)OC)OC)O)OC',
        'CN1CCC2=CC(=C(C3=C2C1CC4=C(C(=C(C=C43)OC)O)OC5=CC=C(C=C5)CC6C7=CC(=C(C=C7CCN6C)O)OC)OC)OC',
#        'CC(C)(CCCC(CC(=O)OC)(C(=O)OC1C2C3=CC4=C(C=C3CCN5C2(CCC5)C=C1OC)OCO4)O)O',
#        'CC(=O)OC1CC(C(=C)C2C1(C(C(C3(C4(COC3(C(=O)CC4C2OC(=O)C)C)C)O)OC(=O)C)OC(=O)C)COC(=O)C5=CC=CC=C5)OC(=O)CC(C6=CC=CC=C6)N(C)C',
#        'CN1CCC2=CC(=C3C=C2C1CC4=CC=C(C=C4)OC5=C(C=CC(=C5)CC6C7=C(O3)C(=C(C=C7CCN6C)OC)OC)OC)OC',
        'CC(C)OC(=O)C(C)NP(=O)(OCC1C(C(C(O1)N2C=CC(=O)NC2=O)(C)F)O)OC3=CC=CC=C3']

In [227]:
id_smiles=['54600918', '10723258', '16215736', '72332', '45375808_Sofosbuvir']

In [9]:
out=pybel.Outputfile(filename='InputMols.mol2',format='mol2',overwrite=True)
for index,smi in enumerate(smiles):
    mol=pybel.readstring(string=smi,format='smiles')
    mol.title='mol_'+str(index)
    mol.make3D('mmff94s')
    mol.localopt(forcefield='mmff94s', steps=500)
    out.write(mol)
out.close()

In [10]:
# Docking box definition
cmd.load(filename='rec_clean_H.pdb',format='pdb',object='prot') 

center,size=getbox(selection='prot',extending=6.0,software='vina')
cmd.delete('all')
print(center)
print(size)

{'center_x': 9.27500057220459, 'center_y': 33.774500131607056, 'center_z': 45.08200025558472}
{'size_x': 80.29599952697754, 'size_y': 71.81100034713745, 'size_z': 77.25000095367432}


In [11]:
random_seed=[]
for i in range(10):
    rs=random.randint(0, 1000000000)
    commande="../bin/smina -r rec_clean_H.pdb -l InputMols.mol2 -o {6}.sdf --center_x {0} --center_y {1} --center_z {2} --size_x {3} --size_y {4} --size_z {5} --exhaustiveness 8 --num_modes 1 --seed {6}".format(center['center_x'], center['center_y'], center['center_z'], size['size_x'], size['size_y'], size['size_z'], rs)
    os.system(commande)
    random_seed.append(str(rs))

In [12]:
print(random_seed)

['459315910', '832878913', '96977782', '67428224', '121376884', '285267341', '779065594', '270778962', '623148856', '940943812']


In [13]:
docking_scores=[]
for i in range(len(random_seed)):
    poses=Chem.SDMolSupplier(str(random_seed[i])+'.sdf',True)
    sc=[]
    for p in list(poses)[::]:
        pose_1=Chem.MolToMolBlock(p)
        #print(p.GetProp('_Name'),'Score: {}'.format(p.GetProp('minimizedAffinity')))
        sc.append(p.GetProp('minimizedAffinity'))
    print(sc)
    docking_scores.append(sc)
    #docking_scores.append(list(map(int, sc)))

['-10.60390', '-9.35279', '-9.22019', '-10.03478', '-9.08740']
['-10.90167', '-10.00183', '-8.61504', '-10.04795', '-9.31175']
['-10.89119', '-11.06563', '-9.47066', '-10.52701', '-8.62330']
['-10.89770', '-11.06850', '-9.06239', '-10.00470', '-8.76418']
['-10.57260', '-10.13529', '-9.53388', '-10.14402', '-8.78278']
['-10.70114', '-11.05660', '-9.48413', '-9.97917', '-8.36812']
['-10.84412', '-11.05853', '-9.44674', '-9.98639', '-8.64276']
['-10.89495', '-11.03850', '-9.37667', '-10.57814', '-8.90967']
['-10.84470', '-11.04604', '-9.30184', '-9.99390', '-8.68922']
['-10.87813', '-11.02379', '-9.12154', '-10.51852', '-9.12714']


In [287]:
df=pd.DataFrame(docking_scores).T
df.columns=random_seed
df = df.astype(float)
df['ids']=id_smiles
first_column = df.pop('ids')
df.insert(0, 'ids', first_column)
df.to_csv("docking_results.csv")

df

,ids,459315910,832878913,96977782,67428224,121376884,285267341,779065594,270778962,623148856,940943812
0,54600918,-10.60390,-10.90167,-10.89119,-10.89770,-10.57260,-10.70114,-10.84412,-10.89495,-10.84470,-10.87813
1,10723258,-9.35279,-10.00183,-11.06563,-11.06850,-10.13529,-11.05660,-11.05853,-11.03850,-11.04604,-11.02379
2,16215736,-9.22019,-8.61504,-9.47066,-9.06239,-9.53388,-9.48413,-9.44674,-9.37667,-9.30184,-9.12154
3,72332,-10.03478,-10.04795,-10.52701,-10.00470,-10.14402,-9.97917,-9.98639,-10.57814,-9.99390,-10.51852
4,45375808_Sofosbuvir,-9.08740,-9.31175,-8.62330,-8.76418,-8.78278,-8.36812,-8.64276,-8.90967,-8.68922,-9.12714


In [288]:
best_scores = df.min(axis=1)
df['best_scores']=best_scores
df

,ids,459315910,832878913,96977782,67428224,121376884,285267341,779065594,270778962,623148856,940943812,best_scores
0,54600918,-10.60390,-10.90167,-10.89119,-10.89770,-10.57260,-10.70114,-10.84412,-10.89495,-10.84470,-10.87813,-10.90167
1,10723258,-9.35279,-10.00183,-11.06563,-11.06850,-10.13529,-11.05660,-11.05853,-11.03850,-11.04604,-11.02379,-11.06850
2,16215736,-9.22019,-8.61504,-9.47066,-9.06239,-9.53388,-9.48413,-9.44674,-9.37667,-9.30184,-9.12154,-9.53388
3,72332,-10.03478,-10.04795,-10.52701,-10.00470,-10.14402,-9.97917,-9.98639,-10.57814,-9.99390,-10.51852,-10.57814
4,45375808_Sofosbuvir,-9.08740,-9.31175,-8.62330,-8.76418,-8.78278,-8.36812,-8.64276,-8.90967,-8.68922,-9.12714,-9.31175


In [289]:
best_value = df['best_scores'].min()
print(best_value)

-11.0685


In [290]:
ids = df.index[df['best_scores'] == best_value].tolist()
print(ids[0])

1


In [292]:
best_conf = df.loc[:, df.columns!='best_scores'].apply(lambda row: row[row==best_value].index.values, axis=1)
for i in range(len(best_conf)):
    if ((best_conf).values[i]!='[]'):
        rs_best_conf=int(best_conf[i])
print(rs_best_conf)

67428224


In [333]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

#view.addModel(open('rec_clean_H.pdb','r').read(),'pdb')
#Prot=view.getModel()
#Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
#view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'white'})

poses=Chem.SDMolSupplier('{0}.sdf'.format(rs_best_conf),True)
#for p in list(poses)[ids[0]]:
p=poses[ids[0]]
pose_1=Chem.MolToMolBlock(p)
print('Molecule id: {}'.format(df['ids'][ids[0]]),'| Random Seed: {}'.format(rs_best_conf),'| Score: {}'.format(p.GetProp('minimizedAffinity')))
#color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
view.addModel(pose_1,'mol')
z= view.getModel()
z.setStyle({},{'stick':{'color':color[0],'radius':0.05,'opacity':0.6}})

view.zoomTo()
view.show()

Molecule id: 10723258 | Random Seed: 67428224 | Score: -11.06850


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [334]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('rec_clean_H.pdb','r').read(),'pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
#view.addSurface(py3Dmol.VDW,{'opacity':0.1,'color':'white'})

poses=Chem.SDMolSupplier('{0}.sdf'.format(rs_best_conf),True)
p=poses[ids[0]]
pose_1=Chem.MolToMolBlock(p)
print('Molecule id: {}'.format(df['ids'][ids[0]]),'| Random Seed: {}'.format(rs_best_conf),'| Score: {}'.format(p.GetProp('minimizedAffinity')))
view.addModel(pose_1,'mol')
z= view.getModel()
z.setStyle({},{'stick':{'color':color[0],'radius':0.05,'opacity':0.6}})

view.zoomTo()
view.show()

Molecule id: 10723258 | Random Seed: 67428224 | Score: -11.06850


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol